In [1]:
import pandas as pd

In [2]:
amend_results = pd.read_csv('amend_results.csv')

In [3]:
rep_results = pd.read_csv('rep_results.csv')

## Sanity check: Each row in amendment results represents a unique precinct, and no precincts are repeated

In [4]:
assert len(amend_results.groupby(['county_name', 'precinct_name'])) == len(amend_results)

## How many precincts are split across multiple House districts?

In [5]:
rep_county_precincts_districts = rep_results \
    .groupby(['county_name', 'precinct_name', 'district']) \
    .agg(
        candidates_count=('candidate_ballot_name', 'count'),
        votes_sum=('yes_votes', 'sum')
    ) \
    .reset_index()

In [6]:
rep_county_precincts_districts

,county_name,precinct_name,district,candidates_count,votes_sum
0,Adair,ABSENTEE,3,2,721
1,Adair,ABSENTEE,4,1,234
2,Adair,BRASHEAR,3,2,0
3,Adair,BRASHEAR,4,1,577
4,Adair,FEDERAL,3,2,0
...,...,...,...,...,...
3782,Wright,LITTLE CREEK,141,1,316
3783,Wright,MANES,141,1,289
3784,Wright,MANSFIELD,141,1,1209
3785,Wright,MOUNTAIN GROVE NORTH,141,1,689


In [7]:
rep_county_precincts = rep_county_precincts_districts \
    .groupby(['county_name', 'precinct_name']) \
    .agg(
        district_count=('district', 'count'),
        votes_sum=('votes_sum', 'sum'),
    ) \
    .reset_index()

In [8]:
rep_county_precincts

,county_name,precinct_name,district_count,votes_sum
0,Adair,ABSENTEE,2,955
1,Adair,BRASHEAR,2,577
2,Adair,FEDERAL,2,0
3,Adair,NORTHEAST 5,2,659
4,Adair,NORTHEAST 6,2,882
...,...,...,...,...
3188,Wright,LITTLE CREEK,1,316
3189,Wright,MANES,1,289
3190,Wright,MANSFIELD,1,1209
3191,Wright,MOUNTAIN GROVE NORTH,1,689


In [9]:
multi_rep_district_precincts = rep_county_precincts[rep_county_precincts['district_count'] > 1]

In [10]:
len(multi_rep_district_precincts)

459

In [11]:
multi_rep_district_precincts.district_count.max()

15

In [12]:
multi_rep_district_precincts.district_count.min()

2

In [13]:
rep_districts = rep_results \
    .groupby('district') \
    .agg(
        votes_sum=('yes_votes', 'sum'),
    )

In [14]:
rep_districts

,votes_sum
district,
1,12695
2,10741
3,11940
4,10386
5,14170
...,...
159,11255
160,14189
161,11885


## Merging precinct-level results for Amendment 1 and House races

In [15]:
amend_rep_merge_outer = amend_results.merge(
    rep_county_precincts_districts,
    on=['county_name', 'precinct_name'],
    how='outer',
    indicator=True,
)

## These precincts appear in amend_results, but not in rep_results

In [16]:
amend_rep_merge_outer[amend_rep_merge_outer['_merge'] == 'left_only']

,county_name,precinct_name,yes_votes,no_votes,district,candidates_count,votes_sum,_merge
217,Boone,INTRASTATE,2.0,2.0,NaN,NaN,NaN,left_only
1034,Greene,INTRASTATE/NEW RESIDENT,6.0,4.0,NaN,NaN,NaN,left_only
1108,Grundy,FEDERAL,0.0,1.0,NaN,NaN,NaN,left_only
1731,Laclede,INTRASTATE,1.0,1.0,NaN,NaN,NaN,left_only
1804,Lincoln,INTRASTATE NEW RESIDENT,0.0,2.0,NaN,NaN,NaN,left_only
2452,Shelby,SHELBYVILLE,164.0,212.0,NaN,NaN,NaN,left_only
2695,St. Louis City,W 02 P 03,384.0,150.0,NaN,NaN,NaN,left_only
2700,St. Louis City,W 02 P 08,470.0,121.0,NaN,NaN,NaN,left_only
3563,St. Louis County,INTRASTATE01,5.0,4.0,NaN,NaN,NaN,left_only
3564,St. Louis County,INTRASTATE02,3.0,4.0,NaN,NaN,NaN,left_only


## These precincts appear in rep_results, but not in amend_results

In [17]:
amend_rep_merge_outer[amend_rep_merge_outer['_merge'] == 'right_only']

,county_name,precinct_name,yes_votes,no_votes,district,candidates_count,votes_sum,_merge
3793,Douglas,PROVISIONAL,NaN,NaN,155.0,1.0,2.0,right_only
3794,Dunklin,CAMPBELL RURAL-UNION,NaN,NaN,152.0,2.0,322.0,right_only
3795,Shelby,SHELBYVILLE/EMDEN/BLKCR/NRTH RIV,NaN,NaN,5.0,2.0,383.0,right_only
3796,St. Louis County,WRITE-IN,NaN,NaN,66.0,1.0,2.0,right_only


In [18]:
amend_rep_merge = amend_results.merge(
    rep_county_precincts_districts,
    on=['county_name', 'precinct_name'],
)

In [19]:
amend_rep_merge

,county_name,precinct_name,yes_votes,no_votes,district,candidates_count,votes_sum
0,Adair,SOUTHWEST 1,342,159,3,2,506
1,Adair,SOUTHWEST 1,342,159,4,1,0
2,Adair,SOUTHEAST 2,688,394,3,2,1094
3,Adair,SOUTHEAST 2,688,394,4,1,0
4,Adair,SOUTHEAST 3,374,156,3,2,527
...,...,...,...,...,...,...,...
3778,Wright,HARTVILLE,300,399,141,1,610
3779,Wright,LITTLE CREEK,164,193,141,1,316
3780,Wright,MANES,115,208,141,1,289
3781,Wright,GROVESPRING,230,358,141,1,532


The table we want:

- District number
- Total number of voters who voted for a rep in that district
- The minimum number of voters who could have voted yes on amendment 1
- The minimum number of voters who could have voted no on amendment 1


In [22]:
county_precinct_amend_min_votes = amend_rep_merge \
    .groupby(['county_name', 'precinct_name']) \
    .filter(lambda x: x['district'].count() == 1)

In [24]:
county_precinct_excluded_votes = amend_rep_merge \
    .groupby(['county_name', 'precinct_name']) \
    .filter(lambda x: x['district'].count() > 1)

In [25]:
county_precinct_excluded_votes

,county_name,precinct_name,yes_votes,no_votes,district,candidates_count,votes_sum
0,Adair,SOUTHWEST 1,342,159,3,2,506
1,Adair,SOUTHWEST 1,342,159,4,1,0
2,Adair,SOUTHEAST 2,688,394,3,2,1094
3,Adair,SOUTHEAST 2,688,394,4,1,0
4,Adair,SOUTHEAST 3,374,156,3,2,527
...,...,...,...,...,...,...,...
3756,Webster,NORTHVIEW - B,269,201,141,1,27
3758,Webster,EAST OZARK & NORTHVIEW A,803,709,137,2,209
3759,Webster,EAST OZARK & NORTHVIEW A,803,709,141,1,1166
3763,Webster,"ABSENTEE, PROVISIONAL & CENTRAL",640,354,137,2,572


In [26]:
district_amend_min_votes = county_precinct_amend_min_votes \
    .groupby('district') \
    .agg(
        min_yes_votes=('yes_votes', 'sum'),
        min_no_votes=('no_votes', 'sum'),
    )

In [27]:
district_amend_min_votes

,min_yes_votes,min_no_votes
district,,
1,6461,6059
2,5941,5979
3,2377,2695
4,5560,5051
5,6969,5732
...,...,...
159,5121,5654
160,3895,4279
161,3618,2713


In [28]:
final_merge = rep_districts \
    .merge(
        district_amend_min_votes,
        on='district',
    )

In [29]:
final_merge['over_half_yes'] = (
    (final_merge.min_yes_votes / final_merge.votes_sum) > 0.5
)

In [30]:
final_merge['over_half_no'] = (
    (final_merge.min_no_votes / final_merge.votes_sum) > 0.5
)

In [31]:
final_merge.to_csv('final_merge.csv')

In [32]:
len(final_merge[final_merge.over_half_no])

11

In [33]:
len(final_merge[final_merge.over_half_yes])

74

In [34]:
final_merge.min_yes_votes.sum()

1132803

In [35]:
final_merge.min_no_votes.sum()

689290